FINAL CONVOLUTION NEURAL NETWORK
The neural net is trained on a combination of provided and original features. The file sensor_geometry.csv contains the (x,y,z) positions of the sensors in the IceCube detector. The file train_meta.parquet provides additional information about each neutrino event. Most importantly, it contains the true azimuth and zenith of the incoming neutrino, which we train against. The neutrino events are stored in files of the form batch_##.parquet, where ## ranges from 1 to 660. Each batch file contains the information on ~200,000 neutrino events. A single neutrino event consists of an arbitrary number of rows in the parquet file, with each row corresponding to a single sensor activation during the neutrino event. 

The all-features.csv file consist of original features derived from the linear regression and clustering computations. We utilize just an initial guess of the azimuth and zenith and an estimate of the number of sensor activation clusters in the event. Additionally we compute a metric of potential scattering in a sensor.

In this network, we encode the position of the sensors by their indices in an appropriately sized three-dimensional tensor with four channels. The four channels are the first time the sensor was activated, the last time it was activated, the total charge detected in this sensor and the above mentioned scatter metric. We used two layers of convolution followed by Max-Pool layers, then flattened the tensor to an array and added in the pre-comptued features. Finally we run three layers of linear layers with ReLu-activation to arrive at our predictions of azimuth and zenith.

In [1]:
#Take Care of the Imports
import pandas as pd
import numpy as np
import os
import torch
import torch.nn as nn
from torch.nn import Linear
from torch.utils.data import Dataset, DataLoader
import torch.autograd.profiler as profiler
import torch.nn.functional as F

In [2]:
'''
Set parent directories for:

sensor_geometry.csv
batch_##.parquet
train_meta.parquet
all-features.csv

respectively. 

'''
pre_dir = "D:/jupyter/erdos-data/"
sensor_geom_dir = "D:/jupyter/erdos-data/"
batch_dir = "D:/jupyter/erdos-data/train/"
meta_dir = "D:/jupyter/erdos-data/"



In [3]:
#Set paths using specified directories. 
pre_path=pre_dir+"all-features.csv"
sensor_geom_path=sensor_geom_dir+"sensor_geometry.csv"
meta_path=meta_dir+"train_meta.parquet"

#Load precompiled features and sensor geometry
pre_feature=pd.read_csv(pre_path)
sensor_geom = pd.read_csv(sensor_geom_path)


#Load metadata, if it is not already loaded. 
try: meta
except NameError: meta=pd.read_parquet(meta_path)

In [4]:
#Set Default Device to CUDA or CPU
#To be called in .to(device) to ensure all pytorch Tensors are on the same device
device = (
    "cuda:0"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
torch.set_default_device(device)
print(f"Using {device} device")

Using cuda:0 device


In [5]:
'''
Torch Dataset Wrapper.
Initialization takes the filename, the sensor geometry file name, the batch ID, and the auxiliary flag.
'''

class NeutrinoDataset(Dataset):
    def __init__(self, batch_filename, sensor_file_name, batch_id, aux):
        
        #save sensor_geometry and meta_data for the given patch
        self.sensor_geom = pd.read_csv(sensor_file_name)
        self.vals_df = meta[meta.batch_id==batch_id]
        
        #Loads the parquet file as a panda dataframe and filter by aux flag
        self.dataframe = pd.read_parquet(batch_filename)
        self.dataframe = self.dataframe[self.dataframe.auxiliary==aux]
        
        #set the number of features per sensor. 
        #Currently, first and last time, cumulative charge, scattering flag, x,y,z.  
        self.num_features = 5160*(3+1+3) 
        
        #Number of neutrino events in the data frame
        self.num_events = self.dataframe.index.nunique()
        
        #Since an event can span multiple rows, save the unique indices, to obtain event_ids.  
        self.unique_indices = np.unique(self.dataframe.index)
        
        #Generate the xyz-lists for the sensor locations. 
        self.x_values, self.y_values, self.z_values=self.get_sensor_pos()
    
    #Return number of Neutrino Events in the Dataset (not the number of rows)
    def __len__(self):
        return self.num_events

    #Get the ith Neutrino Event, based on the ith unique event id
    def __getitem__(self, i):
        
        df = self.dataframe 
        sg = self.sensor_geom
        #Get the event id corresponding to the ith unique index
        event_id=self.unique_indices[i]
        
        #Load the ith neutrino event, and convert to np.array
        event=df.loc[event_id]
        pulse_array = np.array(event)
        
        #Load the metavalues associated to the event
        meta_vals = np.array(
            self.vals_df.loc[self.vals_df['event_id'] == event_id])[0].astype(float)
        
        
        
        # We will fill the following tensor with the data for all sensors
        # There are 4 channels (1st time, last time, charge, and scattering)
        # There are 118 x-coordinate values amongst the sensors
        # There are 117 y-coordinate values amongst the sensors
        # There are 4974 z-coordinate values amongst the sensors, however for memory-saving purposes we block them into 
        # 207-blocks
        
        formated_pulse=torch.zeros(4,22,20,207).to(device)
        
        #Find pulse with largest charge
        loudest=self.loudest_bang(event)
        
        
        for pulse in pulse_array:
            #Gives sensor_id
            sensor_id=pulse[0] 
            
            #Computes (x,y,z) of this sensor_id
            sensor_xyz=self.id_to_xyz(sensor_id)
            
            #Computes the indices in the tensor corresponding to the sensor coordinates
            tensor_ind=self.Tensor_ind_from_coord(sensor_xyz[0], sensor_xyz[1], sensor_xyz[2]) 
            
            #If this sensor has not been activated before, remember that this is the first time
            if( formated_pulse[0,tensor_ind[0],tensor_ind[1],tensor_ind[2]]==0):
                formated_pulse[0,tensor_ind[0],tensor_ind[1],tensor_ind[2]]= pulse[1] - meta_vals[2]
            
            else:
            #Possible last time, will be the last time for the actual last one
                formated_pulse[1,tensor_ind[0],tensor_ind[1],tensor_ind[2]]=pulse[1] - meta_vals[2] # first time
            
            #Add charge
            formated_pulse[2,tensor_ind[0],tensor_ind[1],tensor_ind[2]] += pulse[2]
            
            #Compute the distance between the sensor and the furthest possible cascade from loudest bang.
            scatter=np.linalg.norm( np.array([sensor_xyz[0], sensor_xyz[1], sensor_xyz[2]])-loudest[2])-0.23*(loudest[1]-pulse[1])
            formated_pulse[3,tensor_ind[0],tensor_ind[1],tensor_ind[2]]= scatter
            
            #Include pre-computed azimuth, zenith and number of cluster
            az_t_pred = pre_feature.loc[pre_feature['event_id']==event_id]['az_t_pred'].iloc[0]
            ze_t_pred = pre_feature.loc[pre_feature['event_id']==event_id]['ze_t_pred'].iloc[0]
            num_clusters= pre_feature.loc[pre_feature['event_id']==event_id]['num_clusters'].iloc[0]
            
            
            pre_comp=np.array([az_t_pred,ze_t_pred, num_clusters])
            pre_comp=torch.from_numpy(pre_comp).to(device)
                
            return ([formated_pulse,pre_comp], torch.from_numpy(meta_vals[-2:]).to(device)) #returns the formated pulse and the meta-values
    
    #Function which computes the pulse with maximum charge of a given event and outputs its sensor_id, time and position
    def loudest_bang(self, event):
        charges=event.charge.values
        sensors=event.sensor_id.values
        times=event.time.values
        i=charges.argmax(axis=0)
        sen_max=sensors[i]
        time_max=times[i]
        xyz_from_id=self.id_to_xyz(sen_max)
        max_pos=[xyz_from_id[0], xyz_from_id[1], xyz_from_id[2]]
            
        return [sen_max,time_max,max_pos]
        
        
    #Computes xyz-coordinates based of sensor based on sensor_id   
    def id_to_xyz(self, sen):
        row = tuple(self.sensor_geom.loc[sen][1:4])
        return row
        
        
    #Computes indices of sensor in tensor based on xyz-coordinates   
    def Tensor_ind_from_coord(self, x,y,z):
        tensor_ind_from_coord=[int(x/50),int(y/50),int(z/5)]
        return tensor_ind_from_coord
    
    #Saves all the xyz-coordinates of the sensors (which is event independent)
    def get_sensor_pos(self):
        x_values=np.zeros(0)
        y_values=np.zeros(0)
        z_values=np.zeros(0)
        for i in range(1,5160):
            pos=self.id_to_xyz(i)
            pos=np.array([pos[0],pos[1],pos[2]])
            x_values=np.append(x_values,pos[0])
            y_values=np.append(y_values,pos[1])
            z_values=np.append(z_values,pos[2])
   

        x_values=np.array(sorted(set(x_values)))
        x_values= x_values+570.9 #22 values if downsampel3ed by 50m
        y_values=np.array(sorted(set(y_values)))
        y_values= y_values+521.08 #20 values if downsampeled by 50 m
        z_values=np.array(sorted(set(z_values)))
        z_values=z_values+512.82
        return [x_values, y_values, z_values]

In [6]:
#define our custom loss class
class custom_MAE(nn.Module):
    def __init__(self):
        super(custom_MAE, self).__init__();

    def forward(self, predictions, target):
        loss_value = self.angular_dist_score(predictions, target).to(device)
        return loss_value
    
    #This is the scoring metric provided by Kaggle
    def angular_dist_score(self, predictions, true):
        '''
        calculate the MAE of the angular distance between two directions.
        The two vectors are first converted to cartesian unit vectors,
        and then their scalar product is computed, which is equal to
        the cosine of the angle between the two vectors. The inverse 
        cosine (arccos) thereof is then the angle between the two input vectors
    
        Parameters:
        -----------
    
        az_true : float (or array thereof)
            true azimuth value(s) in radian
        zen_true : float (or array thereof)
            true zenith value(s) in radian
        az_pre : float (or array thereof)
            predicted azimuth value(s) in radian
        zen_pre : float (or array thereof)
            predicted zenith value(s) in radian
    
        Returns:
        --------
    
        dist : float
            mean over the angular distance(s) in radian
        '''
    
        az_true=true[:,0].to(device)
        zen_true=true[:,1].to(device)
        az_pred=predictions[:,0].to(device)
        zen_pred=predictions[:,1].to(device)
    
        if not (torch.all(torch.isfinite(az_true)) and
                torch.all(torch.isfinite(zen_true)) and
                torch.all(torch.isfinite(az_pred)) and
                torch.all(torch.isfinite(zen_pred))):
            raise ValueError("All arguments must be finite")
    
        # pre-compute all sine and cosine values
        sa1 = torch.sin(az_true).to(device)
        ca1 = torch.cos(az_true).to(device)
        sz1 = torch.sin(zen_true).to(device)
        cz1 = torch.cos(zen_true).to(device)
    
        sa2 = torch.sin(az_pred).to(device)
        ca2 = torch.cos(az_pred).to(device)
        sz2 = torch.sin(zen_pred).to(device)
        cz2 = torch.cos(zen_pred).to(device)
    
        # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
        scalar_prod = sz1*sz2*(ca1*ca2 + sa1*sa2) + (cz1*cz2)
    
        # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
        # that might otherwise occure from the finite precision of the sine and cosine functions
        scalar_prod =  torch.clip(scalar_prod, -1, 1)
    
        # convert back to an angle (in radian)
        return torch.mean(torch.abs(torch.arccos(scalar_prod))).to(device)
    

In [7]:
'''
Define the Convolutional Neural Network
This expects a tensor with four features/channels
We apply two cycles of convolutional layers followed by Max-Pool layers
After this we flaten the tensor and add in the pre-computed best-fit azimuth, zenith and number of clusters 
Then we apply three linear layers with ReLu-activation to shrink it down to our azimuth and zenith prediction.
'''

class ConvNet(nn.Module):
    def __init__(self,  use_activation = True ):
        super().__init__()
        
        self.conv1= nn.Conv3d(4,30,4)
        self.pool1=  nn.MaxPool3d(4,4)
        self.conv2= nn.Conv3d(30,20,2)
        self.pool2= nn.MaxPool3d(2,2)
        self.fc1=   nn.Linear(503,100, dtype=float)
        self.fc2=   nn.Linear(100,50, dtype=float)
        self.fc3=   nn.Linear(50,10, dtype= float)
        self.fc4=   nn.Linear(10,2, dtype=float)
        
    
    def forward(self,X):
        x=X[0]
        pre_comp=X[1]
        x=F.tanh(self.conv1(x)) 
        x=self.pool1(x)
        x=F.tanh(self.conv2(x))
        x=self.pool2(x)
        x=torch.flatten(x,1)
        x=torch.cat((x,pre_comp),1)
        x=F.tanh(self.fc1(x))
        x=F.tanh(self.fc2(x))
        x=F.tanh(self.fc3(x))
        x=self.fc4(x)

        return x
    


In [8]:
#Define the Training Loop
def train_loop(dataloader, model, loss_fn, optimizer, epoch, lr, bs):
    size = len(dataloader.dataset)
    # Set the model to training mode
    model=model.train()
    loss_list=np.empty(0)
    for batch, (X, y) in enumerate(dataloader): 
        # Compute preiction and loss
        pred = model(X).to(device)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        #Format training output
        new_loss, current = loss.item(), (batch + 1) * len(X[0])
        loss_list=np.append(loss_list, loss.item())
        if (batch % 1000 == 0 ):
            loss_list=loss_list[-1000:]
            loss=np.mean(loss_list)
            print(f"epoch: {epoch:>2d}, lr: {lr:>2f}, batch_size: {bs:>5d}, loss: {loss:>f}  [{current:>5d}/{size:>5d}]")
        
    return model

In [9]:
def validate_loop(model, val_set):
    #Load the validation data
    validation_dataloader=DataLoader(val_set, batch_size=5, shuffle=False, num_workers=0, generator=torch.Generator(device=device))
    #initialize the loss and number of events
    loss_total = 0
    num = 0
    #Set the model to evaluate
    with torch.no_grad():
        model.eval()
        for batch, (X, y) in enumerate(validation_dataloader):
            # Compute preiction and loss
            pred = model(X)
            loss_total += loss_fn.angular_dist_score(pred,y)
            num +=1
            mean=loss_total/num
        return mean

In [10]:
#Set epoch, batch size, learning rate, loss_fn. 
epoch=20
batch_size=10
learning_rate = 1e-4
loss_fn = custom_MAE()

#Set choice of training batch
batch_id=10
batch_path=batch_dir+"batch_"+str(batch_id)+".parquet"

#Load dataset, setup data splits, and set manual seed
dataset = NeutrinoDataset(batch_path, sensor_geom_path, batch_id, aux=True)
#sub_dataset=torch.utils.data.Subset(dataset, np.arange(100000))
train=torch.utils.data.Subset(dataset, np.arange(150000))
test=torch.utils.data.Subset(dataset, np.arange(150000, 200000))
torch.manual_seed(42)

#Initalize the NN, optimizer, dataloader. 
model = ConvNet()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
train_dataloader = DataLoader(train, batch_size=batch_size, shuffle=False, num_workers=0, generator=torch.Generator(device=device))



In [ ]:

for i in range(epoch):
    model=train_loop(train_dataloader, model, loss_fn, optimizer, i, learning_rate, batch_size)
    mean=validate_loop(model, test)
    print(f"Validation MAE: {mean:>5f}.")


epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.057302  [   10/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.528467  [10010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.429699  [20010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.339994  [30010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.312941  [40010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.275247  [50010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.269443  [60010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.274805  [70010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.247268  [80010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.263310  [90010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.263639  [100010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.236429  [110010/150000]
epoch:  0, lr: 0.000100, batch_size:    10, loss: 1.255628  [120010/150000]
epoch:  0, lr: 0.00010